# Matrices associated with Bernoulli convolutions for Pisot parameters

We set $T_i(x)=\beta x+i$, for some $ \beta\in(1,2)$, and let $r>0$ to be the fixed point of $T_{-1}$ (so that $-r$ is the fixed point of $T_1$). The following is an algorithm for generating the set 

\begin{align*}
    \left\{ T_{\epsilon_n}\circ...\circ T_{\epsilon_1}(0)  :n\in\mathbb{N},(\epsilon_1,...,\epsilon_n)\in\{-1,0,1\}^n\right\}\cap(-r,r),
\end{align*}

in the case where $\beta$ is a Pisot number. The number $\beta$  being Pisot ensures that the set $V$ is  finite. Let $V=\{x_1,...,x_{|V|}\}$. Then we form the $|V|\times|V|$ matrix 

\begin{align*}
    A(i,j)=\begin{cases}
    1/2,\quad \exists \epsilon\in\{-1,1\}:T_\epsilon(x_i)=x_j\\
    1,\quad T_0(x_i)=x_j
    \end{cases},
\end{align*}

which also defines a directed graph where $(x_i,x_j)$ is an edge if and only if $A(i,j)\neq 0$. Then we compute the spectral radius of $A$, the respective leading eigenvector $u$ and the discreet measure $\mu$ that  is supported on $V$ and satisfies $\mu(x_i)=u(x_i)$. We also compute the Wasserstein distance of $m$ from the Lebesgue measure. The motivation for this comes from the fractal geometry of Bernoulli convolutions. The matrix $A$ encodes statistical information of what happens when you zoom in the fractal in a random way. In particular the spectral radius $\rho(A)$ gives a lower bound on the Hausdroff dimension of the respective Bernoulli convolution $\nu_\beta$ by 

\begin{align*}
    \operatorname{dim}_H(\nu_\beta)\geq \min\left\{1,\frac{\log2-\log(\rho(A))}{\log(\beta)}\right\}.
\end{align*}


A final note is that any number in $V$ can be uniquely written in the form $a_{d-1}\beta^{d-1}+...+a_1\beta+a_0$ where $d$ is the algebraic degree of $\beta$. This allows us to represent an element in $V$ by the tuple $(a_0,...,a_{d-1})$. We do our computations in this symbolic representation because this way we can bypass the chaotic properties of the random walk generating $V$. We will start with example of the Pisot number satisfying the equation $\beta^4-\beta^3-1=0$ and in the end we show a table with other examples. 




First we first import the required libraries and input the minimal polynomial. 

In [5]:
import scipy.integrate as inte
import numpy as np
import sympy as sp
from sympy import Symbol,N,solve
from numpy import linalg as LA
import math 

co=[-1,0,0,-1,1]

Now we compute $\beta$.

In [6]:
deg=len(co)-1
x=Symbol('x')
minpol=co[0]
for i in range(1,deg+1):
  minpol=minpol+co[i]*(x**i)
aa=solve(minpol,x)
a=[]
for i in range(deg):
 if sp.im(aa[i])==0 and sp.re(aa[i])>0:
    a.append(aa[i])
beta=float(np.max(list(map(N,a))))

We create a matrix that represents multiplication be $\beta$.

In [7]:
symb=np.zeros((deg,deg))
for j in range(deg):
  for i in range(deg):
    if j<deg-1:
      if i==j+1:
        symb[i][j]=1
    else: 
      symb[i][j]=-co[i]


Below is the code we use to generate our graph. 

In [8]:
bounds1=[0.01,0.01,0.01,0.01,0.01]
ED0=[]
EDR=[]
EDL=[]
ED=[]
SED=[EDL,ED0,EDR]
startpoint=[0]*deg
LED=0


def edge_addition(vertex,map):
    global LED
    t=np.zeros((deg,1))
    for i in range(deg):
      t[i][0]=vertex[i]
    t=np.dot(symb,t)
    t[0][0]=t[0][0]+map
    inside=1
    pol=0
    for k in range(deg):
      pol=pol+t[k][0]*(beta**k)
    if not np.absolute(pol)<1/np.absolute(beta-1)+bounds1[0]:
      inside=0
    if inside ==1:
      newvertex=[0]*deg
      for i in range(deg):
        newvertex[i]=t[i][0]
      new=[vertex,newvertex]
      ED.append(new)
      SED[map+1].append(new)
      LED=LED+1
      

edge_addition(startpoint,-1)
edge_addition(startpoint,0)
edge_addition(startpoint,1) 


sizes=[0,LED]
Lsizes=2
GO=1
while GO==1:
  for i in range(sizes[Lsizes-2],sizes[Lsizes-1]):
    alreadydone=0
    c=0
    while alreadydone==0 and c<=LED-1:
      if ED[i][1]==ED[c][0]:
        alreadydone=1
      c=c+1
    if alreadydone==0:
      edge_addition(ED[i][1],-1)
      edge_addition(ED[i][1],0)
      edge_addition(ED[i][1],1) 
  if sizes[Lsizes-1]==LED:
    GO=0
  sizes.append(LED)
  Lsizes=Lsizes+1

We define a function that projects a tuple of integers $(a_0,...,a_{d-1})$ back to the real number $a_{d-1}\beta^{d-1}+...+a_1\beta+a_0$.

In [9]:
def projection(vertex):
  global beta
  pol=0
  t=vertex
  for k in range(deg):
    pol=pol+t[k]*(beta**k)
  return(pol)

We generate the matrix $A$. 

In [10]:
Vertices=[]
for i in range(LED):
  Vertices.append(ED[i][0])
  Vertices.append(ED[i][1]) 
Vertices=np.unique(Vertices, axis=0)
Vertices=list(Vertices)
for i in range(len(Vertices)):
  Vertices[i]=list(Vertices[i])
Vertices.sort(key=projection)


gm=np.zeros((len(Vertices),len(Vertices)))
for i in range(len(EDR)):
  row=Vertices.index(EDR[i][0])
  column=Vertices.index(EDR[i][1])
  gm[row][column]=1/2
for i in range(len(ED0)):
  row=Vertices.index(ED0[i][0])
  column=Vertices.index(ED0[i][1])
  gm[row][column]=1
for i in range(len(EDL)):
  row=Vertices.index(EDL[i][0])
  column=Vertices.index(EDL[i][1])
  gm[row][column]=1/2


We get the spectral radious and the elading eigenevector. 

In [11]:
eigs=LA.eig(np.transpose(gm))
maxind=np.argmax(eigs[0].real,0)
rho=(eigs[0].real)[maxind]
v=eigs[1][:,maxind]
v=(v*(1/(np.sum(v)))).real

And we caclulate the Wasserstein distance. 

In [12]:
points=list(map(projection,Vertices))
dist=0
for i in range(len(Vertices)):
  left=-1/(beta-1)
  for j in range(i):
    left=left+v[j]*(2/(beta-1))
  right=left+v[i]*(2/(beta-1))
  aa=float(points[i])
  dist=dist+((beta-1)/2)*(inte.quad(lambda x: np.absolute(x-aa),left,right)[0])

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:9: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  if __name__ == '__main__':


Now we print the number $\beta$, the size of $A$, the lower bound we get from the spectral radius and the Wasserstein distance. 

In [13]:
print(beta)
print(len(Vertices))
print(-(math.log(rho)-math.log(2))/math.log(beta))
print(dist*(beta-1))

1.3802775690976141
1257
0.9999894980962348
0.014903190610378599


Below is the data for some other polynomials,

|$\text{Polynomial}$|$\beta$|$\text{Bound}$|$W_1(\mu,\operatorname{Leb})$|$\text{Matrix size}$|
| --- | --- | --- | --- | --- |
|${\bf x^3-x^2-x-1}$ | $1.8393$ | $0.96422$ | $0.13925$| $7$ |
|$x^3-x^2-1$ | $1.4656$ | $0.999116$ | $0.0547178$|$51$|
|$x^3-x-1$ | $1.3247$ | $0.99999$ | $0.0286671$| $181$|
|${\bf x^4-x^3-x^2-x-1}$ | $1.9276$ | $0.973329$ | $0.187067$| $9$|
|$x^4-x^3-1$ | $1.3803$ | $0.999989$ | $0.0149032$| $1257$|
|${\bf x^5-x^4-x^3-x^2-x-1}$ | $1.9659$ | $0.983565$  | $0.222569$| $11$|
|$x^5-x^4-x^3-x^2-1$ | $1.8885$ | $0.982269$ | $0.0803806$| $745$ |
|$x^5-x^4-x^3-x^2+1$ | $1.7785$ | $0.995758$ | $0.0246573$| $951$| 
|$x^5-x^4-x^3-1$ | $1.7049$ | $0.993043$  |$0.0356598$| $339$ |
|$x^5-x^4-x^3-x-1$ | $1.8124$ | $0.982434$ | $0.0571201$| $351$ |
|$x^5-x^4-x^3+x^2-1$ | $1.4432$ | $0.999982$ | $0.00782515$| $5423$|
|$x^5-x^4-x^2-1$ | $1.5702$ | $0.999862$ | $0.0195581$| $847$|
|$x^5-x^3-x^2-x-1$ | $1.5342$ | $0.999833$ | $0.00890312$| $2651$|